# Scrape data on reservoir

In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

import requests
from bs4 import BeautifulSoup

import re

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/ilenapeng/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
/var/folders/9x/ttxn_gwj0592mr6h8s5y29980000gn/T/ipykernel_50578/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
driver.get("https://valleywateralert.org/scvwd/historicdata/qrgi_historicdata_setup.php")

In [4]:
# Select Stevens Creek Reservoir
Select(driver.find_element(By.ID, 'single_sensor')).select_by_value("4009")

In [5]:
# Select End Month - December
Select(driver.find_element(By.ID, 'month_single_sensor1')).select_by_value("12")

# Select End Day - 1st
Select(driver.find_element(By.ID, 'date_single_sensor1')).select_by_value("01")

# Select End Year - 2021
Select(driver.find_element(By.ID, 'year_single_sensor1')).select_by_value("2021")

In [6]:
# Select Start Month - January
Select(driver.find_element(By.ID, 'month_single_sensor2')).select_by_value("01")

# Select Start Day - 15th
Select(driver.find_element(By.ID, 'date_single_sensor2')).select_by_value("15")

# Select Start Year - 2022
Select(driver.find_element(By.ID, 'year_single_sensor2')).select_by_value("2022")

In [7]:
# Select Time Interval
Select(driver.find_element(By.ID, 'time_interval_single_sensor')).select_by_value("360")

In [8]:
# Click "Get Historical Data"
driver.find_element(By.XPATH, '//*[@id="CenterContent"]/div[3]/form/input').click()

In [9]:
doc = BeautifulSoup(driver.page_source)

In [10]:
data = []
for row in doc.find_all('tr')[2:]:
    each_row = {}
    each_row['DateTime']=row.find('td', {'id':'DateandTime'}).string
    each_row['Elevation']=row.find('td', {'id':'ReservoirElevation'}).string
    each_row['Storage']=row.find('td', {'id':'ReservoirStorage'}).string
    each_row['Capacity']=row.find('td', {'id':'ReservoirCapacity'}).string
    data.append(each_row)
len(data)

184

In [11]:
reservoir_df = pd.DataFrame(data)
reservoir_df.head()

,DateTime,Elevation,Storage,Capacity
0,11/30/2021 06:00:00,486.6,318.8,10.4
1,11/30/2021 12:00:00,486.6,318.8,10.4
2,11/30/2021 18:00:00,486.6,318.8,10.4
3,12/01/2021 00:00:00,486.6,318.8,10.4
4,12/01/2021 06:00:00,486.6,318.8,10.4


In [12]:
# remove november 30th rows
reservoir_df = reservoir_df.iloc[3: , :]

reservoir_df.to_csv("reservoir.csv", index=False)